# Parquet Content-Defined Chunking

Apache Parquet is a columnar storage format that is widely used in the data engineering community. 

As Hugging Face hosts nearly 11PB of datasets with Parquet files alone accounting for over 2.2PB of that storage, optimizing Parquet storage is of high priority.
Hugging Face has introduced a new storage layer called [Xet](https://huggingface.co/blog/xet-on-the-hub) that leverages content-defined chunking to efficiently deduplicate chunks of data reducing storage costs and improving download/upload speeds.

While Xet is format agnostic, Parquet's layout and column-chunk (data page) based compression can produce entirely different byte-level representations for data with minor changes, leading to suboptimal deduplication performance. To address this, the Parquet files should be written in a way that minimizes the byte-level differences between similar data, which is where content-defined chunking (CDC) comes into play.

Let's explore the performance benefits of the new Parquet CDC feature used alongside Hugging Face's Xet storage layer.

### Note about required pyarrow version

The parquet content-defined chunking feature hasn't been released yet, so we need to install a nightly build of `pyarrow`:

## Prepare the data to experiment with

For demonstration purposes, we will use a manageable sized subset of [OpenOrca](https://huggingface.co/datasets/Open-Orca/OpenOrca) dataset.

In [30]:
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq
from huggingface_hub import hf_hub_download


# download the dataset from Hugging Face Hub into local cache
path = hf_hub_download(
    repo_id="Open-Orca/OpenOrca", 
    filename="3_5M-GPT3_5-Augmented.parquet", 
    repo_type="dataset"
)

# read the cached parquet file into a PyArrow table 
orca = pq.read_table(path)

# augment the table some additional columns
orca = orca.add_column(
    orca.schema.get_field_index("question"),
    "question_length",
    pc.utf8_length(orca["question"])
)
orca = orca.add_column(
    orca.schema.get_field_index("response"),
    "response_length",
    pc.utf8_length(orca["response"])
)

# limit the table to the first 100,000 rows 
table = orca[:100_000]

# take a look at the first 5 rows of the table
table[:5].to_pandas()

,id,system_prompt,question_length,question,response_length,response
0,t0.1791914,You are an AI assistant that follows instructi...,291,Q:The exercise is to decide whether the questi...,5,True.
1,flan.2203053,"You are a helpful assistant, who always provid...",132,Sentence 1: There is no need. \n\nSentence 2: ...,136,"Yes, if the first sentence is true, then the s..."
2,flan.1943030,"You are a helpful assistant, who always provid...",392,On his oak mantelpiece are a drinking bowl fro...,293,"We cannot conclude that the sentence ""The Ebol..."
3,t0.870962,You are an AI assistant that follows instructi...,2121,Question: Read the following paragraph and ext...,153,"The full name of the person whose version of ""..."
4,t0.314926,You are an AI assistant that follows instructi...,1703,I have a test where I am given the following a...,146,The Rova compound remained largely closed to t...


### Upload the table as a Parquet file to Hugging Face Hub

Since [pyarrow>=21.0.0](https://github.com/apache/arrow/pull/45089) we can use Hugging Face URIs in the `pyarrow` functions to directly read and write parquet (and other file formats) files to the Hub using the `hf://` URI scheme.

In [ ]:
# import pyarrow.parquet as pq

# # Write the table to the Hugging Face Hub
# pq.write_table(table, "hf://datasets/kszucs/pq/orca.parquet")

We can see that the table has been uploaded entirely as new data because it is not known to the Xet storage layer yet. Now read it back as a `pyarrow` table:

In [ ]:
# table = pq.read_table("hf://datasets/kszucs/pq/orca.parquet")
# len(table)

Note that all `pyarrow` functions that accept a file path also accept a Hugging Face URI, like [pyarrow datasets](https://arrow.apache.org/docs/python/dataset.html), 
[CSV functions](https://arrow.apache.org/docs/python/generated/pyarrow.csv.read_csv.html), [incremental Parquet writer](https://arrow.apache.org/docs/python/generated/pyarrow.parquet.ParquetWriter.html) or reading only the parquet metadata:

In [ ]:
# pq.read_metadata("hf://datasets/kszucs/pq/orca.parquet")

## Different Use Cases for Parquet Deduplication

To demonstrate the effectiveness of the content-defined chunking feature, we will try out how it performs in case of:
1. Re-uploading exact copies of the table
2. Adding/removing columns from the table
3. Changing column types in the table
4. Appending new rows and concatenating tables
5. Inserting / deleting rows in the table
6. Change row-group size of the table
7. Change file-level data splitting


### 1. Re-uploading an Exact Copies of the Table

While this use case sounds trivial, traditional file systems do not deduplicate files resulting in full re-upload and re-download of the data. In contrast, a system utilizing content-defined chunking can recognize that the file content is identical and avoid unnecessary data transfer.

In [ ]:
# pq.write_table(table, "hf://datasets/kszucs/pq/orca-copy.parquet")

We can see that no new data has been uploaded, and the operation was instantaneous. Now let's see what happens if we upload the the same file again but to a different repository:


In [ ]:
# pq.write_table(table, "hf://datasets/kszucs/pq-copy/orca-copy-again.parquet")

The upload was instantaneous since deduplication works across repositories as well. This is a key feature of the Xet storage layer, allowing efficient data sharing and collaboration. 

### 2. Adding and Removing Columns from the Table

First write out the original and changed tables to local parquet files to see their sizes:

In [36]:
table_with_new_columns = table.add_column(
    table.schema.get_field_index("response"),
    "response_short",
    pc.utf8_slice_codeunits(table["response"], 0, 10)
)
table_with_removed_columns = table.drop(["response"])
    
pq.write_table(table, "/tmp/original.parquet")
pq.write_table(table_with_new_columns, "/tmp/with-new-columns.parquet")
pq.write_table(table_with_removed_columns, "/tmp/with-removed-columns.parquet")

In [37]:
!ls -lah /tmp/*.parquet

-rw-r--r--  1 kszucs  wheel    91M Jul 20 13:21 /tmp/original.parquet
-rw-r--r--  1 kszucs  wheel    92M Jul 20 13:21 /tmp/with-new-columns.parquet
-rw-r--r--  1 kszucs  wheel    67M Jul 20 13:21 /tmp/with-removed-columns.parquet


Now upload them to Hugging Face to see how much data is actually transferred:

In [ ]:
# pq.write_table(table_with_new_columns, "hf://datasets/kszucs/pq/orca-added-columns.parquet")

We can see that only the new columns and the new parquet metadata placed in the file's footer were uploaded, while the original data was not transferred again. This is a huge benefit of the content-defined chunking feature, as it allows us to efficiently add new columns without transferring the entire dataset again. 

Same applies to removing columns, as we can see below:

In [ ]:
# pq.write_table(table_with_removed_columns, "hf://datasets/kszucs/pq/orca-removed-columns.parquet")

We can also visualize the deduplication between the two parquet files using the deduplication estimation tool:

In [40]:
visualize(table, {"with-new-columns": table_with_new_columns})


#### Parquet Deduplication for With-new-columns
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![with-new-columns Vanilla](temp-none-with-new-columns-nocdc.parquet.png) | ![with-new-columns Vanilla](temp-zstd-with-new-columns-nocdc.parquet.png) | ![with-new-columns Vanilla](temp-snappy-with-new-columns-nocdc.parquet.png) |



┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃              ┃             ┃             ┃            ┃            ┃   Compressed ┃             ┃    Compressed ┃
┃ Title        ┃ Compression ┃ Parquet CDC ┃ Total Size ┃ Chunk Size ┃   Chunk Size ┃ Dedup Ratio ┃   Dedup Ratio ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ with-new-co… │ none        │ nocdc       │  298.3 MiB │  149.7 MiB │     95.3 MiB │         50% │           32% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-new-co… │ snappy      │ nocdc       │  183.2 MiB │   91.9 MiB │     92.3 MiB │         50% │           50% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-new-co… │ zstd        │ nocdc       │  127.1 MiB │   63.9 MiB │     64.2 MiB │         50% │           50% │
└──────────────┴─────────────┴─────────────┴────────────┴────────────┴──────────────┴─────────────┴───────────────┘

Adding two new columns mean that we have unseen data pages which must be transferred (highlighted in red), but the rest of the data remains unchanged (highlighted in green), so it is not transferred again. Note the small red area in the footer metadata which almost always changes as we modify the parquet file.

In [41]:
visualize(table, {"with-removed-columns": table_with_removed_columns})


#### Parquet Deduplication for With-removed-columns
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![with-removed-columns Vanilla](temp-none-with-removed-columns-nocdc.parquet.png) | ![with-removed-columns Vanilla](temp-zstd-with-removed-columns-nocdc.parquet.png) | ![with-removed-columns Vanilla](temp-snappy-with-removed-columns-nocdc.parquet.png) |



┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃              ┃             ┃             ┃            ┃            ┃   Compressed ┃             ┃    Compressed ┃
┃ Title        ┃ Compression ┃ Parquet CDC ┃ Total Size ┃ Chunk Size ┃   Chunk Size ┃ Dedup Ratio ┃   Dedup Ratio ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ with-remove… │ none        │ nocdc       │  256.3 MiB │  148.9 MiB │     94.6 MiB │         58% │           37% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-remove… │ snappy      │ nocdc       │  158.7 MiB │   91.4 MiB │     91.8 MiB │         58% │           58% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-remove… │ zstd        │ nocdc       │  110.2 MiB │   63.4 MiB │     63.6 MiB │         57% │           58% │
└──────────────┴─────────────┴─────────────┴────────────┴────────────┴──────────────┴─────────────┴───────────────┘

Since we are removing entire columns we can only see changes in the footer metadata, all the other columns remain unchanged and already existing in the storage layer, so they are not transferred again.

### 3. Changing Column Types in the Table

Another common use case is changing the column types in the table e.g. to reduce the storage size or to optimize the data for specific queries. Let's change the `score` column from `float64` to `float32` and see how much data is transferred:

In [42]:
# first make the table much smaller by removing the largest column
# this will highlight the change in the heatmap much better
table_without_text = table_with_new_columns.drop(["question", "response"])

# cast the question_length column to int64
table_with_casted_column = table_without_text.set_column(
    table_without_text.schema.get_field_index("question_length"),
    "question_length",
    table_without_text["question_length"].cast("int64")
)

Again, we can see that only the new column and the updated parquet metadata were uploaded. Now visualize the deduplication heatmap:

In [43]:
visualize(table_without_text, {"with-casted-column10": table_with_casted_column})


#### Parquet Deduplication for With-casted-column10
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![with-casted-column10 Vanilla](temp-none-with-casted-column10-nocdc.parquet.png) | ![with-casted-column10 Vanilla](temp-zstd-with-casted-column10-nocdc.parquet.png) | ![with-casted-column10 Vanilla](temp-snappy-with-casted-column10-nocdc.parquet.png) |



┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃              ┃             ┃             ┃            ┃            ┃   Compressed ┃             ┃    Compressed ┃
┃ Title        ┃ Compression ┃ Parquet CDC ┃ Total Size ┃ Chunk Size ┃   Chunk Size ┃ Dedup Ratio ┃   Dedup Ratio ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ with-casted… │ none        │ nocdc       │    5.0 MiB │    2.9 MiB │      2.1 MiB │         57% │           41% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-casted… │ snappy      │ nocdc       │    3.4 MiB │    2.0 MiB │      2.0 MiB │         57% │           58% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-casted… │ zstd        │ nocdc       │    2.6 MiB │    1.5 MiB │      1.5 MiB │         56% │           56% │
└──────────────┴─────────────┴─────────────┴────────────┴────────────┴──────────────┴─────────────┴───────────────┘

The first red block indicates the new column that was added, while the second red block indicates the updated metadata in the footer. The rest of the data remains unchanged and is not transferred again.

### 4. Appending New Rows and Concatenating Tables

We are going to append new rows by concatenating another slice of the original dataset to the table. 

In [44]:
table = orca[:100_000]
next_10k_rows = orca[100_000:110_000]
appended = pa.concat_tables([table, next_10k_rows])

assert len(appended) == 110_000

Now check that only the new rows are being uploaded since the original data is already known to the Xet storage layer:

In [ ]:
# pq.write_table(table_with_appended_rows, "hf://datasets/kszucs/pq/orca-appended-rows.parquet")

In [46]:
visualize(table, {"with-appended-rows": appended})


#### Parquet Deduplication for With-appended-rows
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![with-appended-rows Vanilla](temp-none-with-appended-rows-nocdc.parquet.png) | ![with-appended-rows Vanilla](temp-zstd-with-appended-rows-nocdc.parquet.png) | ![with-appended-rows Vanilla](temp-snappy-with-appended-rows-nocdc.parquet.png) |



┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃              ┃             ┃             ┃            ┃            ┃   Compressed ┃             ┃    Compressed ┃
┃ Title        ┃ Compression ┃ Parquet CDC ┃ Total Size ┃ Chunk Size ┃   Chunk Size ┃ Dedup Ratio ┃   Dedup Ratio ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ with-append… │ none        │ nocdc       │  312.5 MiB │  164.9 MiB │    104.9 MiB │         53% │           34% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-append… │ snappy      │ nocdc       │  191.8 MiB │  101.5 MiB │    101.8 MiB │         53% │           53% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-append… │ zstd        │ nocdc       │  133.0 MiB │   70.5 MiB │     70.8 MiB │         53% │           53% │
└──────────────┴─────────────┴─────────────┴────────────┴────────────┴──────────────┴─────────────┴───────────────┘

Since each column gets new data, we can see multiple red strides. This is due to the actual parquet file specification where whole columns are layed out after each other (within each row group). Note the large read area at the bottom which is the new data for the `text` column. 

### 5. Inserting / Deleting Rows in the Table

Here comes the difficult part as insertions and deletions are shifting the existing rows which lead to different columns chunks or data pages in the parquet nomenclature. Since each data page is compressed separately, even a single row insertion or deletion can lead to a completely different byte-level representation starting from the edited row(s) to the end of the parquet file. 

This parquet specific problem cannot be solved by the Xet storage layer alone, the parquet file itself needs to be written in a way that minimizes the data page differences even if there are inserted or deleted rows. 

Let's try to use the existing mechanism and see how it performs.

In [47]:
table = orca[:100_000]

# remove 4k rows from two places 
table_with_deleted_rows = pa.concat_tables([
    orca[:15_000], 
    orca[18_000:60_000],
    orca[61_000:100_000]
])

# add 1k rows at the first third of the table
table_with_inserted_rows = pa.concat_tables([
    orca[:10_000],
    orca[100_000:101_000],
    orca[10_000:50_000],
    orca[101_000:103_000],
    orca[50_000:100_000],
])

assert len(table) == 100_000
assert len(table_with_deleted_rows) == 96_000
assert len(table_with_inserted_rows) == 103_000

In [ ]:
# pq.write_table(table_original, "hf://datasets/kszucs/pq/orca-inserted-rows.parquet")

In [ ]:
# pq.write_table(table_with_deleted_rows, "hf://datasets/kszucs/pq/orca-deleted-rows.parquet")

In [50]:
from de import visualize 

visualize(table, {"deleted-rows": table_with_deleted_rows, "inserted-rows": table_with_inserted_rows})


#### Parquet Deduplication for Deleted-rows
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![deleted-rows Vanilla](temp-none-deleted-rows-nocdc.parquet.png) | ![deleted-rows Vanilla](temp-zstd-deleted-rows-nocdc.parquet.png) | ![deleted-rows Vanilla](temp-snappy-deleted-rows-nocdc.parquet.png) |




#### Parquet Deduplication for Inserted-rows
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![inserted-rows Vanilla](temp-none-inserted-rows-nocdc.parquet.png) | ![inserted-rows Vanilla](temp-zstd-inserted-rows-nocdc.parquet.png) | ![inserted-rows Vanilla](temp-snappy-inserted-rows-nocdc.parquet.png) |



┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃              ┃             ┃             ┃            ┃            ┃    Compressed ┃             ┃   Compressed ┃
┃ Title        ┃ Compression ┃ Parquet CDC ┃ Total Size ┃ Chunk Size ┃    Chunk Size ┃ Dedup Ratio ┃  Dedup Ratio ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ deleted-rows │ none        │ nocdc       │  291.8 MiB │  174.5 MiB │     111.0 MiB │         60% │          38% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼───────────────┼─────────────┼──────────────┤
│ deleted-rows │ snappy      │ nocdc       │  179.1 MiB │  166.0 MiB │     166.6 MiB │         93% │          93% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼───────────────┼─────────────┼──────────────┤
│ deleted-rows │ zstd        │ nocdc       │  124.2 MiB │  115.3 MiB │     115.8 MiB │         93% │          93% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼───────────────┼─────────────┼──────────────┤
│ inserted-ro… │ none        │ nocdc       │  302.1 MiB │  180.3 MiB │     114.7 MiB │         60% │          38% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼───────────────┼─────────────┼──────────────┤
│ inserted-ro… │ snappy      │ nocdc       │  185.5 MiB │  176.7 MiB │     177.3 MiB │         95% │          96% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼───────────────┼─────────────┼──────────────┤
│ inserted-ro… │ zstd        │ nocdc       │  128.6 MiB │  122.8 MiB │     123.3 MiB │         96% │          96% │
└──────────────┴─────────────┴─────────────┴────────────┴────────────┴───────────────┴─────────────┴──────────────┘

We can see that the deduplication ratio has dropped significantly, and the deduplication heatmaps show that the compressed parquet files are quite different from each other. This is due to the fact that the inserted and deleted rows have shifted the existing rows, leading to different data pages in the parquet file. Since each data page is compressed separately, even a single row insertion or deletion can lead to a completely different byte-level representation starting from the edited row(s) to the end of the parquet file. 

We can solve this problem by writing parquet files with a new [pyarrow feature called content-defined chunking (CDC)](https://github.com/apache/arrow/pull/45360). This feature ensures that the columns are consistently getting chunked into data pages based on their content, similarly how the Xet storage layer deduplicates data but applied on the logical values of the columns before any serialization or compression happens. 

The feature can be enabled by passing `use_content_defined_chunking=True` to the `write_parquet` function:

```python
import pyarrow.parquet as pq

pq.write_table(table, "hf://user/repo/filename.parquet", use_content_defined_chunking=True)
```

Let's visualize the deduplication difference before and after using the Parquet CDC feature:

In [51]:
visualize(table, {"with-deleted-rows": table_with_deleted_rows, "with-inserted-rows": table_with_inserted_rows}, 
          with_cdc=True)


#### Parquet Deduplication for With-deleted-rows
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![with-deleted-rows Vanilla](temp-none-with-deleted-rows-nocdc.parquet.png) | ![with-deleted-rows Vanilla](temp-zstd-with-deleted-rows-nocdc.parquet.png) | ![with-deleted-rows Vanilla](temp-snappy-with-deleted-rows-nocdc.parquet.png) |
| CDC Parquet | ![with-deleted-rows CDC](temp-none-with-deleted-rows-cdc.parquet.png) | ![with-deleted-rows CDC](temp-zstd-with-deleted-rows-cdc.parquet.png) | ![with-deleted-rows CDC](temp-snappy-with-deleted-rows-cdc.parquet.png) |




#### Parquet Deduplication for With-inserted-rows
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![with-inserted-rows Vanilla](temp-none-with-inserted-rows-nocdc.parquet.png) | ![with-inserted-rows Vanilla](temp-zstd-with-inserted-rows-nocdc.parquet.png) | ![with-inserted-rows Vanilla](temp-snappy-with-inserted-rows-nocdc.parquet.png) |
| CDC Parquet | ![with-inserted-rows CDC](temp-none-with-inserted-rows-cdc.parquet.png) | ![with-inserted-rows CDC](temp-zstd-with-inserted-rows-cdc.parquet.png) | ![with-inserted-rows CDC](temp-snappy-with-inserted-rows-cdc.parquet.png) |



┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃              ┃             ┃             ┃            ┃            ┃   Compressed ┃             ┃    Compressed ┃
┃ Title        ┃ Compression ┃ Parquet CDC ┃ Total Size ┃ Chunk Size ┃   Chunk Size ┃ Dedup Ratio ┃   Dedup Ratio ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ with-delete… │ none        │ cdc         │  292.2 MiB │  154.1 MiB │     98.0 MiB │         53% │           34% │
│ with-delete… │ none        │ nocdc       │  291.8 MiB │  174.5 MiB │    111.0 MiB │         60% │           38% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-delete… │ snappy      │ cdc         │  179.6 MiB │   99.4 MiB │     99.8 MiB │         55% │           56% │
│ with-delete… │ snappy      │ nocdc       │  179.1 MiB │  166.0 MiB │    166.6 MiB │         93% │           93% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-delete… │ zstd        │ cdc         │  125.0 MiB │   69.4 MiB │     69.7 MiB │         56% │           56% │
│ with-delete… │ zstd        │ nocdc       │  124.2 MiB │  115.3 MiB │    115.8 MiB │         93% │           93% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-insert… │ none        │ cdc         │  302.5 MiB │  157.7 MiB │    100.3 MiB │         52% │           33% │
│ with-insert… │ none        │ nocdc       │  302.1 MiB │  180.3 MiB │    114.7 MiB │         60% │           38% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-insert… │ snappy      │ cdc         │  186.0 MiB │  100.8 MiB │    101.2 MiB │         54% │           54% │
│ with-insert… │ snappy      │ nocdc       │  185.5 MiB │  176.7 MiB │    177.3 MiB │         95% │           96% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ with-insert… │ zstd        │ cdc         │  129.4 MiB │   70.4 MiB │     70.6 MiB │         54% │           55% │
│ with-insert… │ zstd        │ nocdc       │  128.6 MiB │  122.8 MiB │    123.3 MiB │         96% │           96% │
└──────────────┴─────────────┴─────────────┴────────────┴────────────┴──────────────┴─────────────┴───────────────┘

Since the proof of the pudding is in the eating, let's actually upload the tables using the content-defined chunking parquet feature and see how much data is transferred:

In [52]:
# pq.write_table(table, "hf://datasets/kszucs/pq/finemath-1m.parquet", use_content_defined_chunking=True)

In [53]:
# pq.write_table(table, "hf://datasets/kszucs/pq/finemath-1m.parquet", use_content_defined_chunking=True)

### 6. Using different Row-Group Sizes

There are cases depending on the reader/writer contraints where larger or smaller row-group sizes might be beneficial. The parquet writer implementations use fixed-sized row-groups by default, in case of pyarrow the default is 1Mi rows. Dataset writers may change to reduce the row-group size in order to improve random access performance or to reduce the memory footprint of the reader application.

Chainging the row-group size will shift rows between row-groups, shifting values between data pages, so we have a similar problem as with inserting or deleting rows. Let's compare the deduplication performance between different row-group sizes using the parquet CDC feature:

In [54]:
orca[:5].to_pandas()

,id,system_prompt,question_length,question,response_length,response
0,t0.1791914,You are an AI assistant that follows instructi...,291,Q:The exercise is to decide whether the questi...,5,True.
1,flan.2203053,"You are a helpful assistant, who always provid...",132,Sentence 1: There is no need. \n\nSentence 2: ...,136,"Yes, if the first sentence is true, then the s..."
2,flan.1943030,"You are a helpful assistant, who always provid...",392,On his oak mantelpiece are a drinking bowl fro...,293,"We cannot conclude that the sentence ""The Ebol..."
3,t0.870962,You are an AI assistant that follows instructi...,2121,Question: Read the following paragraph and ext...,153,"The full name of the person whose version of ""..."
4,t0.314926,You are an AI assistant that follows instructi...,1703,I have a test where I am given the following a...,146,The Rova compound remained largely closed to t...


In [55]:
from de import visualize

# pick a larger subset of the dataset to have enough rows for the row group size tests
table = orca[2_000_000:3_000_000]

cases = {
    "small-row-groups2": (table, {"row_group_size": 128 * 1024}),
    "medium-row-groups2": (table, {"row_group_size": 256 * 1024}),
}
visualize(table, cases, with_cdc=True)


#### Parquet Deduplication for Small-row-groups2
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![small-row-groups2 Vanilla](temp-none-small-row-groups2-nocdc.parquet.png) | ![small-row-groups2 Vanilla](temp-zstd-small-row-groups2-nocdc.parquet.png) | ![small-row-groups2 Vanilla](temp-snappy-small-row-groups2-nocdc.parquet.png) |
| CDC Parquet | ![small-row-groups2 CDC](temp-none-small-row-groups2-cdc.parquet.png) | ![small-row-groups2 CDC](temp-zstd-small-row-groups2-cdc.parquet.png) | ![small-row-groups2 CDC](temp-snappy-small-row-groups2-cdc.parquet.png) |




#### Parquet Deduplication for Medium-row-groups2
    
| Variant | No Compression | Zstd Compression  | Snappy Compression |
|---------|----------------|-------------------|--------------------|
| Vanilla Parquet | ![medium-row-groups2 Vanilla](temp-none-medium-row-groups2-nocdc.parquet.png) | ![medium-row-groups2 Vanilla](temp-zstd-medium-row-groups2-nocdc.parquet.png) | ![medium-row-groups2 Vanilla](temp-snappy-medium-row-groups2-nocdc.parquet.png) |
| CDC Parquet | ![medium-row-groups2 CDC](temp-none-medium-row-groups2-cdc.parquet.png) | ![medium-row-groups2 CDC](temp-zstd-medium-row-groups2-cdc.parquet.png) | ![medium-row-groups2 CDC](temp-snappy-medium-row-groups2-cdc.parquet.png) |



┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃              ┃             ┃             ┃            ┃            ┃   Compressed ┃             ┃    Compressed ┃
┃ Title        ┃ Compression ┃ Parquet CDC ┃ Total Size ┃ Chunk Size ┃   Chunk Size ┃ Dedup Ratio ┃   Dedup Ratio ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ medium-row-… │ none        │ cdc         │    2.9 GiB │    1.5 GiB │    958.2 MiB │         50% │           32% │
│ medium-row-… │ none        │ nocdc       │    2.9 GiB │    1.6 GiB │   1018.1 MiB │         54% │           34% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ medium-row-… │ snappy      │ cdc         │    1.8 GiB │  932.0 MiB │    935.4 MiB │         51% │           51% │
│ medium-row-… │ snappy      │ nocdc       │    1.8 GiB │    1.2 GiB │      1.2 GiB │         67% │           67% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ medium-row-… │ zstd        │ cdc         │    1.2 GiB │  649.2 MiB │    651.6 MiB │         51% │           51% │
│ medium-row-… │ zstd        │ nocdc       │    1.2 GiB │  850.4 MiB │    853.6 MiB │         67% │           67% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ small-row-g… │ none        │ cdc         │    2.9 GiB │    1.5 GiB │    966.8 MiB │         51% │           32% │
│ small-row-g… │ none        │ nocdc       │    2.9 GiB │    1.7 GiB │      1.0 GiB │         57% │           36% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ small-row-g… │ snappy      │ cdc         │    1.8 GiB │  945.4 MiB │    948.8 MiB │         51% │           52% │
│ small-row-g… │ snappy      │ nocdc       │    1.8 GiB │    1.4 GiB │      1.4 GiB │         80% │           81% │
├──────────────┼─────────────┼─────────────┼────────────┼────────────┼──────────────┼─────────────┼───────────────┤
│ small-row-g… │ zstd        │ cdc         │    1.2 GiB │  658.6 MiB │    661.0 MiB │         52% │           52% │
│ small-row-g… │ zstd        │ nocdc       │    1.2 GiB │ 1019.6 MiB │   1023.5 MiB │         80% │           81% │
└──────────────┴─────────────┴─────────────┴────────────┴────────────┴──────────────┴─────────────┴───────────────┘

### 7. Using Different File-Level Splitting

Datasets often split into multiple files to improve parallelism and random access. Parquet CDC combined with the Xet storage layer can efficiently deduplicate data across multiple files even if the data is split at different boundaries. 

Let's write out the dataset with three different file-level splitting then compare the deduplication performance:

In [56]:
from pathlib import Path

def write_dataset(table, base_dir, num_shards, **kwargs):
    """Simple utility to write a pyarrow table to multiple Parquet files."""
    # ensure that directory exists
    base_dir = Path(base_dir)
    base_dir.mkdir(parents=True, exist_ok=True)
    # split and write the table into multiple files
    rows_per_file = len(table) / num_shards
    for i in range(num_shards):
        start = i * rows_per_file
        end = min((i + 1) * rows_per_file, len(table))
        shard = table.slice(start, end - start)
        path = base_dir / f"part-{i}.parquet"
        pq.write_table(shard, path, **kwargs)

In [60]:
from de import estimate

table = orca

write_dataset(table, "orca3-cdc", num_shards=3, use_content_defined_chunking=True)
write_dataset(table, "orca8-cdc", num_shards=8, use_content_defined_chunking=True)

estimate("orca3-nocdc/*.parquet", "orca8-nocdc/*.parquet")


thread '<unnamed>' panicked at src/lib.rs:24:48:
called `Option::unwrap()` on a `None` value


PanicException: called `Option::unwrap()` on a `None` value

In [ ]:
# TODO(kszucs): maybe add the stats plot for the HF repo starts dataset revisions